This notebook is ran in a docker container where the project directory (i.e. same directory as README.md) is located in `/code`, which is set below. If you run locally you'll need to set the path of your project directory accordingly.

The `load_dotenv` function below loads all the variables found in the `.env` file as environment variables. You must have a `.env` file located in the project directory containing your OpenAI API key, in the following format.

```
OPENAI_API_KEY=sk-...
```

---

In [ ]:
%cd /code

In [ ]:
from dotenv import load_dotenv
load_dotenv()

---

# Examples

## Find relevant documents to include in prompt

In [ ]:
texts = [
    'This is a document. It has information related to the question I want to ask.',
    'The codeword is `flibberwump`; the answer is `hanzo`.',
    'Here is another document.',
]
question = "What is the answer for the codeword `flibberwump`?"

In [ ]:
from llm_chain.base import Document
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.models import OpenAIEmbeddings

# create a document index (i.e. vector database) and add the text from above.
document_index = ChromaDocumentIndex(
    embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'),
)
document_index.add(docs=[Document(content=x) for x in texts])

In [ ]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIChat
from llm_chain.prompt_templates import DocSearchTemplate

doc_template = DocSearchTemplate(doc_index=document_index, n_docs=1)

# A chain is simply a collection of callables where the output of the previous callable matches
# the input to the next callable.
# Below, the input to the `DocSearchTemplate` is a string (the question) and the output is a
# string (the prompt); and the input to `OpenAIChat` is a string (the prompt).
# Question (str) -> Prompt (str) -> Answer (str)
chain = Chain(links=[
    DocSearchTemplate(doc_index=document_index, n_docs=1),
    OpenAIChat(model_name='gpt-3.5-turbo'),
])
response = chain()
response

In [ ]:
# the chain tracks the usage across any object that has `total_tokens` and `total_cost` properties
print(f"Tokens: {chain.total_tokens:,}")
print(f"Cost: ${chain.total_cost:.6}")

In [ ]:
# you can see the individual costs for the embeddings and the chat
# The embeddings model has 2 records in its history; 
# 1 to embed the original docs and the other to embed the question passed into the chain
for record in chain.history:
    print(record)

In [ ]:
# The chat has model has 1 record in its history
chat_model = chain[1]
print(f"prompt: {chat_model._history[0].prompt}")
print(f"response: {chat_model._history[0].response}")
print(f"cost: {chat_model._history[0].cost}")

---

URL -> doc -> text-splitter -> list[docs] -> vector-db (embeddings) -> None

query -> search -> 

---

In [ ]:
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.models import OpenAIEmbeddings
from llm_chain.chains import Chain
from llm_chain.models import OpenAIChat
from llm_chain.prompt_templates import DocSearchTemplate

# create a document index (i.e. vector database) and add the text from above.
document_index = ChromaDocumentIndex(
    embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'),
)

In [ ]:
def load_text():
    return [
        'This is a document. It has information related to the question I want to ask.',
        'The codeword is `flibberwump`; the answer is `hanzo`.',
        'Here is another document.',
    ]

In [ ]:
# <nothing> -> list[str]
# list[str] -> None
# <ignored> -> str
# str -> str
# str -> str
chain = Chain(links=[
    load_text,
    lambda texts: document_index.add(docs=[Document(content=x) for x in texts]),
    lambda _: "What is the answer for the codeword `flibberwump`?",
    DocSearchTemplate(doc_index=document_index, n_docs=1),
    OpenAIChat(model_name='gpt-3.5-turbo'),
])
response = chain()
response

In [ ]:
# the chain tracks the usage across any object that has `total_tokens` and `total_cost` properties
print(f"Tokens: {chain.total_tokens:,}")
print(f"Cost: ${chain.total_cost:.6}")

In [ ]:
for record in chain.history:
    print(record)

---

In [ ]:
from llm_chain.tools import html_page_loader, split_documents

doc = html_page_loader(url='https://python.langchain.com/en/latest/modules/agents.html')
doc.content

In [ ]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain
from llm_chain.prompt_templates import DocSearchTemplate

chat_model_name = 'gpt-3.5-turbo'
emb_model_name = 'text-embedding-ada-002'

# document_index is used to store and retrieve documents scraped from the URL defined below
# the __call__ function calls add() or search() based on input
document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name=emb_model_name))
# prompt_template is retrieves the most relevant docs and stuffs them into the prompt
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
chat = OpenAIChat(model_name=chat_model_name)
# converts a string to a list containing a single Document object
text_to_docs = lambda x: [Document(content=x.replace('\n', ' '))]

# questions for ChatGPT; each link in the chain must be a callable
# The first question uses the context from the url via the prior links
ask_question_1 = lambda _: "What is a langchain `Agent`?"
# the second question uses the answer from ChatGPT as part of the prompt
question_2 = lambda x: f'Summarize the following in less than 10 words: "{x}"'

# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    html_page_loader,
    text_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    ask_question_1,
    prompt_template,
    chat,
    question_2,
    chat,
])

response = chain('https://python.langchain.com/en/latest/modules/agents.html')
response

In [ ]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

In [ ]:
for record in chain.history:
    print(record)

---

# Web Search via DuckDuckGo

In [ ]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import duckduckgo_search, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain
from llm_chain.prompt_templates import DocSearchTemplate

# document_index is used to store and retrieve documents scraped from the URL defined below
# the __call__ function calls add() or search() based on the input
document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'))
# prompt_template is retrieves the most relevant docs and stuffs them into the prompt
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
chat = OpenAIChat(model_name='gpt-3.5-turbo')

# for each url, extracts text, cleans, returns doc
def search_results_to_docs(results: list[dict]) -> list[Document]:
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]

question_1 = "What is a langchain `Agent`?"

# questions for ChatGPT; each link in the chain must be a callable
# The first question uses the context from the url via the prior links
ask_question_1 = lambda _: question_1
# the second question uses the answer from ChatGPT as part of the prompt
ask_question_2 = lambda x: f'Summarize the following in less than 10 words: "{x}"'

# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    ask_question_1,
    prompt_template,
    chat,
    ask_question_2,
    chat,
])

response = chain(question_1)
response

In [ ]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

In [ ]:
for record in chain.history:
    print(record)

In [ ]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

---

In [ ]:
initial_question.value

In [ ]:
class CallableClass:
    def __init__(self):
        self.value = None
    
    def __call__(self, value=None):
        if value is not None:
            self.value = value
        return self.value


# Create an instance of CallableClass
my_callable = CallableClass()

# Call the instance without passing a value
result1 = my_callable()  # Returns: None
print(result1)

# Call the instance and pass a value
my_callable("Hello, World!")

# Call the instance again without passing a value
result2 = my_callable()  # Returns: "Hello, World!"
print(result2)


In [9]:
from llm_chain.chains import Chain
from llm_chain.models import OpenAIEmbeddings, OpenAIChat
from llm_chain.tools import duckduckgo_search, html_page_loader, split_documents
from llm_chain.indexes import ChromaDocumentIndex
from llm_chain.base import Document
from llm_chain.chains import Chain, Value
from llm_chain.prompt_templates import DocSearchTemplate

# the __call__ function calls add() or search() based on the input
document_index = ChromaDocumentIndex(embeddings_model=OpenAIEmbeddings(model_name='text-embedding-ada-002'))
# prompt_template is retrieves the most relevant docs and stuffs them into the prompt
prompt_template = DocSearchTemplate(doc_index=document_index, n_docs=2)
# OpenAI Chat model
chat = OpenAIChat(model_name='gpt-3.5-turbo')

# for each url, extracts text, cleans, returns doc
def search_results_to_docs(results: list[dict]) -> list[Document]:
    return [Document(content=html_page_loader(x['href']).replace('\n', ' ')) for x in results]


initial_question = Value()
# questions for ChatGPT; each link in the chain must be a callable
# The first question uses the context from the url via the prior links
# the second question uses the answer from ChatGPT as part of the prompt
question_2 = lambda x: f'Summarize the following in less than 20 words: "{x}"'

# each link is a callable where the output of one link is the input to the next
chain = Chain(links=[
    initial_question,
    duckduckgo_search,
    search_results_to_docs,
    split_documents,  # defaults to chunk-size of 500
    document_index,  # __call__ function calls add() or search() based on input
    initial_question,
    prompt_template,
    chat,
    question_2,
    chat,
])

response = chain("What is a langchain agent?")
response

'A LangChain Agent drives decision-making, accesses tools, and builds adaptive applications with context-specific responses.'

In [10]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost: $0.0019
Tokens: 8,319


In [11]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

## MESSAGE:  0

### PROMPT:
Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
f LangChain to build advanced language model applications that are adaptable, efficient, and capable of handling complex use cases.What is a LangChain Agent?A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses. They are especially useful when there's an unknown chain of interactions that de

 LangChain enables chains to interact with external data sources to gather data for the generation step. For example, it can help with summarizing long texts or answering questions using specific data sources.Agents: An agent lets an LLM make decisions about actions, take those actions, check the results, and keep going until the job's done. LangChain provides a standard interface for agents, a variety of agents to choose from, and examples of end-to-end agents.Memory: LangChain has a standard i
```

Here is the question:

What is a langchain agent?

### RESPONSE:
 > A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses.

---

## MESSAGE:  1

### PROMPT:
Summarize the following in less than 20 words: "A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses."

### RESPONSE:
 > A LangChain Agent drives decision-making, accesses tools, and builds adaptive applications with context-specific responses.

---

In [12]:
response = chain("What is a langchain document loader?")
response

'A LangChain Document Loader is a versatile tool that loads text from various sources and transforms data for language models.'

In [13]:
print(f"Cost: ${chain.total_cost:.4f}")
print(f"Tokens: {chain.total_tokens:,}")

Cost: $0.0048
Tokens: 16,495


In [14]:
from IPython.display import display, Markdown
for index, record in enumerate(chain.message_history):
    display(Markdown(f"## MESSAGE:  {index}"))
    display(Markdown(f"### PROMPT:\n{record.prompt.strip()}"))
    display(Markdown(f"### RESPONSE:\n > {record.response.strip()}"))
    display(Markdown("---"))

## MESSAGE:  0

### PROMPT:
Answer the question at the end of the text as truthfully and accurately as possible, based on the following information provided.

Here is the information:

```
f LangChain to build advanced language model applications that are adaptable, efficient, and capable of handling complex use cases.What is a LangChain Agent?A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses. They are especially useful when there's an unknown chain of interactions that de

 LangChain enables chains to interact with external data sources to gather data for the generation step. For example, it can help with summarizing long texts or answering questions using specific data sources.Agents: An agent lets an LLM make decisions about actions, take those actions, check the results, and keep going until the job's done. LangChain provides a standard interface for agents, a variety of agents to choose from, and examples of end-to-end agents.Memory: LangChain has a standard i
```

Here is the question:

What is a langchain agent?

### RESPONSE:
 > A LangChain Agent is an entity that drives decision-making in the framework. It has access to a set of tools and can decide which tool to call based on the user's input. Agents help build complex applications that require adaptive and context-specific responses.

---

## MESSAGE:  1

...

---

## MESSAGE:  3

### PROMPT:
Summarize the following in less than 20 words: "A LangChain Document Loader is a versatile tool that can handle various data formats and transform them into a standard structure that language models can easily process. It loads text from various sources and is used in language modeling tasks."

### RESPONSE:
 > A LangChain Document Loader is a versatile tool that loads text from various sources and transforms data for language models.

---

---


# OpenAI Chat

## Simple example showing history and usages/costs

In [ ]:
from llm_chain.models import OpenAIChat

chat = OpenAIChat(model_name='gpt-3.5-turbo', temperature=0)
response = chat("Hi, my name is Shane.")
response

In [ ]:
# the model object tracks usage/cost data across all messages  
def print_usage(model: OpenAIChat):
    usage = f"""
    Total Cost: ${model.total_cost:.6f}
    Total Tokens: {model.total_tokens:,}
    Total Prompt Tokens: {model.total_prompt_tokens:,}
    Total Response Tokens: {model.total_response_tokens:,}
    """
    print(usage)

In [ ]:
print_usage(model=chat)

In [ ]:
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

In [ ]:
# the `history` property contains a list of `MessageMetaData` objects for each message (i.e.
# prompt & response) which contains usage/cost data for that message.
for record in chat.history:
    print(record)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
response = chat("Do you remember my name?")
response

In [ ]:
for record in chat.history:
    print(record)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
# You can get the last MessageMetaData via: 
print(f"MessageMetaData: {chat.previous_message}")
# Or you can get the last prompt/response
print(f"previous prompt: {chat.previous_prompt}")
print(f"previous response: {chat.previous_response}")

In [ ]:
print_usage(model=chat)

---

## Memory

The `OpenAIChat` model has a `memory_strategy` parameter and takes a `MemoryBuffer` class. A `MemoryBuffer` class is a callable that takes a `list[MessageMetaData]` (i.e. from the `model.history` property) and also returns a `list[MessageMetaData]` serving as the model's memory (i.e. a list containing the messages that will be sent to the model along with the new prompt). This allows the end user to easily define a memory strategy of their own (e.g. keep the first message and the last `n` messages).

One Example of a `MemoryBuffer` is a `MemoryBufferMessageWindow` class where you can specify the last `n` messages that you want to keep.

In [ ]:
from llm_chain.models import OpenAIChat
from llm_chain.memory import MemoryBufferMessageWindow

chat = OpenAIChat(
    model_name='gpt-3.5-turbo',
    temperature=0,
    memory_strategy=MemoryBufferMessageWindow(last_n_messages=0),  # no memory
)
response = chat("Hi, my name is Shane.")
response

In [ ]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
response = chat("Do you remember my name?")
response

In [ ]:
# we still have access to the full history, but the ChatGPT didn't use any of it.
chat._history

In [ ]:
# you can also see the exact messages sent to ChatGPT
chat._previous_memory

In [ ]:
# NOTE: since we created a new OpenAIChat object, the costs/usage are reset
print_usage(model=chat)

---